# OPEN FOOD NOTEBOOK

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
#import folium
import re
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns

#from google.cloud import translate
#import pycountry
#import emoji

#translate_client = translate.Client()

from py_translator import Translator
# from googletrans import Translator

We need to create a data folder where the .csv file will be stored and also a maps folder where .html maps will be stored

In [2]:
data_folder = './data/'
maps_folder = './maps/'

In [3]:
filename = 'en.openfoodfacts.org.products.csv'
countryfile = 'wikipedia-iso-country-codes.csv'

In [4]:
using_col = [
    "product_name",
    "generic_name",
    "quantity",
    "brands",
    "brands_tags",
    "categories",
    "categories_tags",
    "categories_en",
    "manufacturing_places",
    "manufacturing_places_tags",
    "labels",
    "labels_tags",
    "labels_en",
    "purchase_places",
    "countries",
    "countries_tags",
    "countries_en",
    "ingredients_text",
    "allergens",
    "allergens_en",
    "traces",
    "traces_tags",
    "traces_en",
    "nutrition_grade_uk",
    "nutrition_grade_fr",
    "main_category",
    "main_category_en",
    "energy_100g",
    "energy-from-fat_100g",
    "fat_100g",
    "saturated-fat_100g",
    "trans-fat_100g",
    "cholesterol_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "alcohol_100g",
    "calcium_100g",
    "iron_100g",
    "carbon-footprint_100g",
    "nutrition-score-fr_100g",
    "nutrition-score-uk_100g",
    "glycemic-index_100g"
]

In [5]:
food_df = pd.read_csv(data_folder + filename, 
                      sep='\t',
                      header=0,
                      usecols = using_col,
                      quotechar='"', 
                      low_memory=False)

In [6]:
print("The types of the data set are: \n", format(food_df.dtypes))
print ("The total size of the data set is:", format(food_df.shape) )
food_df.head(5)

The types of the data set are: 
 product_name                  object
generic_name                  object
quantity                      object
brands                        object
brands_tags                   object
categories                    object
categories_tags               object
categories_en                 object
manufacturing_places          object
manufacturing_places_tags     object
labels                        object
labels_tags                   object
labels_en                     object
purchase_places               object
countries                     object
countries_tags                object
countries_en                  object
ingredients_text              object
allergens                     object
allergens_en                  object
traces                        object
traces_tags                   object
traces_en                     object
nutrition_grade_uk           float64
nutrition_grade_fr            object
main_category                 object
main_

,product_name,generic_name,quantity,brands,brands_tags,categories,categories_tags,categories_en,manufacturing_places,manufacturing_places_tags,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.8,1.40,0.551181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,2.04,0.803150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,0.42,0.165354,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning Data

## Filling in missing product_name
This section deals with NaN values for `product_name`. If it does not have a `product_name`, the `generic_name` was used. If neither field was filled, then a combination of `brands` and `categories_en`/`main_category` was used.

In [7]:
# remove rows where the columns we are interested in are all null
food_df = food_df.dropna(subset=using_col, how='all')

print("Number of rows w/missing product_name: {}".format(len(food_df) - food_df['product_name'].count()))

Number of rows w/missing product_name: 25881


In [8]:
def get_name(row):
    if pd.isnull(row['product_name']):
        if pd.isnull(row['generic_name']):
            if pd.isnull(row['main_category_en']) & pd.isnull(row['categories_en']) & pd.isnull(row['brands']):
                return
            else:
                category_name = row['main_category_en']
                if pd.isnull(category_name):
                    category_name = row['categories_en']
                return "{} {}".format(row['brands'], category_name)
        else:
            return row['generic_name']
    else:
        return row['product_name']
    
df = food_df.copy()

df['product_name'] = df.apply(
    lambda x: get_name(x),
    axis = 1
)

In [9]:
print("Number of rows w/missing product_name after modifications: {}".format(len(df) - df['product_name'].count()))

Number of rows w/missing product_name after modifications: 22458


In [10]:
# removing the columns that we no longer need
used_col = [
    'generic_name',
    'main_category',
    'main_category_en',
    'brands',
    'brands_tags',
    'categories',
    'categories_tags',
    'categories_en',
]

df = df.drop(used_col, axis=1)

## Filling in Missing Values for Country
This section deals with the missing values for `countries_en`. The `countries_en` column represents the countries where the product is sold. This column is important for our analysis because we want to analyze how viable it is to live in each country based off one's dietary restrictions.

In order to fix these missing values, we decided to first fill the column with values from `purchase_places`, then `manufacturing_places`. We decided to use `purchase_places` because if it was purchased in a certain country, obviously it means the product is sold there as well. As for `manufacturing_places`, we assumed that it is most likely that a product manufactured in a country would be sold there as well.

In [11]:
print("Number of rows w/missing countries_en: {}".format(len(df) - df['countries_en'].count()))

Number of rows w/missing countries_en: 459


In [12]:
def translate_country(row):
    if pd.isnull(row['countries_en']):
        alt_country = None
        if pd.notna(row['purchase_places']):
            alt_country = row['purchase_places']
        elif pd.notna(row['manufacturing_places']):
            alt_country = row['manufacturing_places']
            
        # got value from purchase_places or manufacturing_places
        if (not alt_country is None) and pd.notna(alt_country):
            translator = Translator()
            try:
                en_alt_country = translator.translate(text=alt_country, dest='en')
                if not en_alt_country is None:
                    return en_alt_country.text
            except Exception as e:
                return alt_country
            
        return alt_country
    else:
        return row['countries_en']
    
df_1 = df.copy()

df_1['countries_en'] = df_1.apply(
    lambda x: translate_country(x),
    axis = 1
)

In [13]:
print("Number of rows w/missing countries_en: {}".format(len(df_1) - df_1['countries_en'].count()))

Number of rows w/missing countries_en: 322


In [14]:
# drop rows without country
df_1 = df_1.dropna(subset=['countries_en'])


print("Number of rows w/multiple countries: {}".format(
    len(df_1[df_1['countries_en'].str.contains(',')])))

print("Number of total rows: {}".format(len(df_1)))

Number of rows w/multiple countries: 28626
Number of total rows: 693524


In [15]:
# remove columns that are no longer needed
used_col = [
    'purchase_places',
    'manufacturing_places',
    'manufacturing_places_tags',
    'countries_tags',
    'countries',
]

df_1 = df_1.drop(used_col, axis = 1)

In [16]:
# shows that some countries_en are lists
df_1[df_1['countries_en'].notnull() & df_1['countries_en'].str.contains(',')][['countries_en', 'product_name']].head()

,countries_en,product_name
173,"France,United States",Lion Peanut x2
196,"France,United States",Moelleux
205,"France,United States",Pack de 2 Twix
290,"France,United States",Root Beer
426,"France,Germany",Fuzetea schwarzer Tee Zitrone


In [17]:
# converts the list substring into an array
df_2 = df_1.copy()

df_2['countries_en'] = df_2.apply(
    lambda x: [x.strip() for x in x['countries_en'].split(',')],
    axis = 1
)

# [x.strip() for x in my_string.split(',')]

In [18]:
# shows that the countries has been properly split
df_2[df_2.index == 173][['product_name', 'countries_en']]

,product_name,countries_en
173,Lion Peanut x2,"[France, United States]"


Next, this subsection deals with standardizing the countries for each product. First, we notice that some products have more than one country in their `countries_en` field. In this case, we seperate/explode each country in the `countries_en` field so that each country has its own row for that item. Next, we join the countries with their respective country code.

In [19]:
# map the countries_en to country codes
country_df = pd.read_csv(data_folder + countryfile, 
                         sep=',',
                         header=0,
                         usecols=['English short name lower case', 'Alpha-2 code'],
                         quotechar='"')
# rename columns
country_df.rename(columns={
    'Alpha-2 code':'country_code',
    'English short name lower case': 'country_name'
    }, inplace=True)

# country_df['country_name'] = country_df.apply(
#     lambda x: x['country_name'].split(',')[0] if ',' in x['country_name'] else x['country_name'],
#     axis=1
# )

country_df['country_name'] = country_df.apply(
    lambda x: x['country_name'].lower(),
    axis=1
)

country_df.head()

,country_name,country_code
0,afghanistan,AF
1,åland islands,AX
2,albania,AL
3,algeria,DZ
4,american samoa,AS


In [20]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

df_3 = df_2.copy()
df_3 = explode(df_3,'countries_en')

In [21]:
# see how the explode function created another row because there were two countries for Lion Peanut x2
df_3[df_3['product_name'].notna() & df_3['product_name'].str.contains('Lion Peanut x2')]

,product_name,quantity,labels,labels_tags,labels_en,countries_en,ingredients_text,allergens,allergens_en,traces,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
173,Lion Peanut x2,NaN,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,Lion Peanut x2,NaN,NaN,NaN,NaN,United States,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_4 = df_3.copy()
# rename countries_en to country_name
df_4.rename(columns={'countries_en':'country_name'}, inplace=True)

df_4['country_name'] = df_4.apply(
    lambda x: x['country_name'].lower(),
    axis = 1
)

In [23]:
def notAssigned(df_sample):
    not_assigned = df_sample[df_sample['country_name'].notna() & df_sample['country_code'].isna()]

    print("Number of unassigned items is: {}".format(len(not_assigned)))
    print("The important values are: ")
    print(not_assigned['country_name'].value_counts())

In [24]:
df_5 = df_4.copy()

df_5 = df_5.merge(country_df, how='left')
    
notAssigned(df_5)

Number of unassigned items is: 4566
The important values are: 
russia                                                                                2203
en                                                                                     477
fr:deutschland                                                                         229
taiwan                                                                                 227
vietnam                                                                                107
de:allemagne                                                                            92
ch:suisse                                                                               82
european union                                                                          80
south korea                                                                             67
fr:schweiz                                                                              49
republic of macedonia      

We decided to fix the country codes with the highest frequency, since the importance/effect of fixing the lower values will decrease as we descend through the list.

In [25]:
# changing Russian Federation to russia
country_df['country_name'][country_df['country_code'] == 'RU'] = 'russia'

# changing Korea, Republic of to south korea
country_df['country_name'][country_df['country_code'] == 'KR'] = 'south korea'

# changing Macedonia, the former Yugoslav Republic of to republic of macedonia
country_df['country_name'][country_df['country_code'] == 'MK'] = 'republic of macedonia'

# changing Taiwan, Province of China to taiwan
country_df['country_name'][country_df['country_code'] == 'TW'] = 'taiwan'

# changing Viet Nam to vietnam
country_df['country_name'][country_df['country_code'] == 'VN'] = 'vietnam'


In [26]:
translator = Translator()

def parseTranslate(x):
    if (':') in x['country_name']:
        info_ = x['country_name'].split(':')
        if len(info_) == 2:
            try:
                translate_country = translator.translate(text=info_[1], dest='en').text
                return translate_country.lower()
            except Exception:
                return info_[1]
        
    return x['country_name']

df_6 = df_4.copy()

df_6['country_name'] = df_6.apply(
    lambda x: parseTranslate(x),
    axis = 1
)

In [30]:
df_6 = df_6.merge(country_df, how='left')

def best_match(country_df, row):
    if pd.isnull(row['country_code']):
        countries = list(country_df['country_name'])
        for country in countries:
            if row['country_name'] in country:
                return country

    return row['country_name']

df_6['country_name'] = df_6.apply(
    lambda x: best_match(country_df, x),
    axis = 1
)

Number of unassigned items is: 1921
The important values are: 
argentina                                                                             498
deutschland                                                                           234
allemagne                                                                              96
suisse                                                                                 85
european union                                                                         80
schweiz                                                                                49
frankreich                                                                             35
categories completed                                                                   34
product name completed                                                                 34
brands completed                                                                       34
ingredients completed                

In [37]:
df_7 = df_6.drop(['country_code'], axis=1)

df_ = df_6.merge(country_df, how='left')

# notAssigned(df_6)

df_6[df_6['country_name'] == 'iran, islamic republic of']

,product_name,quantity,labels,labels_tags,labels_en,country_name,ingredients_text,allergens,allergens_en,traces,...,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,country_code
178907,Mask,1,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238550,"Moser Roth Edel Bitter, 70% Kakao",NaN,NaN,NaN,NaN,"iran, islamic republic of",{Sojas. Kano auch Anteile von Mitch. Hasetnuss...,NaN,NaN,NaN,...,0.050,0.019685,NaN,NaN,NaN,NaN,22.0,23.0,NaN,NaN
533683,None,NaN,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603455,Nando's,NaN,vegan,"en:vegetarian,en:vegan","Vegetarian,Vegan","iran, islamic republic of",NaN,NaN,NaN,NaN,...,1.798,0.707874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608010,None,NaN,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608011,Kalleh Labneh Cheese,750 g,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608016,None,NaN,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608028,None,NaN,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608034,قامتت,NaN,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608037,بسکویت,ها,NaN,NaN,NaN,"iran, islamic republic of",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
